# P3 - Movie Genre Classification

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

### Objective

Classify a movie genre based on its plot.
Source: https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

- Input: movie plot
- Output: Probability of the movie belong to each genre

### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 

### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.
See https://arxiv.org/abs/1702.01992

## Sample Submission

In [5]:
# Librerías

import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

import keras
from keras import backend as K

import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

#pip install livelossplot
from livelossplot import PlotLossesKeras
%matplotlib inline

import nltk
from nltk.corpus import stopwords

from xgboost import XGBClassifier


In [6]:
df_train = pd.read_csv('DataSet/dataTraining.csv', encoding='UTF-8', index_col=0)
df_test = pd.read_csv('Dataset/dataTesting.csv', encoding='UTF-8', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'DataSet/dataTraining.csv'

In [ ]:
df_train.head()

In [ ]:
df_train['rating'] = df_train['rating'].astype(int)
df_train.info()

In [ ]:
df_test.head()

In [ ]:
df_test.info()

### Create count vectorizer


In [15]:
english_stopwords = stopwords.words('english')

vect = CountVectorizer(max_features=1000 , stop_words=english_stopwords, ngram_range=(1, 1))
X_dtm = vect.fit_transform(df_train['plot'])

(7895, 1000)

In [16]:
print(vect.get_feature_names())

['able', 'about', 'accepts', 'accident', 'accidentally', 'across', 'act', 'action', 'actor', 'actress', 'actually', 'adam', 'adult', 'adventure', 'affair', 'after', 'again', 'against', 'age', 'agent', 'ago', 'agrees', 'air', 'alan', 'alex', 'alice', 'alien', 'alive', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'an', 'and', 'angeles', 'ann', 'anna', 'another', 'any', 'anyone', 'anything', 'apartment', 'appears', 'are', 'area', 'army', 'around', 'arrested', 'arrive', 'arrives', 'art', 'artist', 'as', 'asks', 'assigned', 'assistant', 'at', 'attack', 'attempt', 'attempts', 'attention', 'attracted', 'aunt', 'away', 'baby', 'back', 'bad', 'band', 'bank', 'bar', 'based', 'battle', 'be', 'beautiful', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'begin', 'begins', 'behavior', 'behind', 'being', 'believe', 'believes', 'beloved', 'ben', 'best', 'better', 'between', 'beyond', 'big', 'bill', 'billy', 'birthday', 'black

### Create y

In [17]:
df_train['genres'] = df_train['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(df_train['genres'])


In [18]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [19]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [20]:
X_train

<5289x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 267819 stored elements in Compressed Sparse Row format>

In [21]:
X_test

<2606x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 130363 stored elements in Compressed Sparse Row format>

In [22]:
y_train_genres

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [23]:
y_test_genres

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Train multi-class multi-label model

In [24]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))

In [25]:
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                     random_state=42))

In [26]:
y_pred_genres = clf.predict_proba(X_test)

In [27]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.764422401477475

### Predict the testing dataset

In [28]:
X_test_dtm = vect.transform(df_test['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)


In [29]:
res = pd.DataFrame(y_pred_test_genres, index=df_test.index, columns=cols)

In [30]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.123756,0.103867,0.022934,0.029989,0.366989,0.119883,0.038821,0.516515,0.071927,0.104363,...,0.025385,0.065428,0.000011,0.359025,0.052039,0.008811,0.018552,0.183363,0.021250,0.020424
4,0.129210,0.089728,0.023649,0.083562,0.357282,0.199651,0.092806,0.512687,0.063970,0.067027,...,0.024091,0.060564,0.001466,0.144207,0.059189,0.012512,0.019936,0.208792,0.041162,0.019134
5,0.202882,0.139104,0.015613,0.065951,0.346519,0.484273,0.014112,0.579468,0.066952,0.111810,...,0.022031,0.290749,0.000000,0.380282,0.111593,0.001286,0.046793,0.473437,0.047256,0.036477
6,0.156802,0.115972,0.018002,0.078629,0.341305,0.147089,0.011142,0.589117,0.066103,0.064168,...,0.109783,0.101384,0.000000,0.200932,0.102672,0.001218,0.060744,0.240969,0.089068,0.021008
7,0.186878,0.153099,0.043560,0.032572,0.293750,0.250824,0.011410,0.418683,0.078710,0.155671,...,0.022231,0.088035,0.000000,0.218954,0.246036,0.003571,0.023050,0.256546,0.020322,0.016667


In [31]:
res.to_csv('pred_genres_text_RF.csv', index_label='ID')

ModuleNotFoundError: No module named 'pandas.io.formats.csvs'

# Red Neuronal

In [34]:
# Crear el vocabulario
X= df_train['plot'].tolist()

In [35]:
# For vocabulary only the intersec characters is used to avoid issues with data collection
voc = set(''.join(X))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

In [36]:
vocabulary

{'r': 1,
 '1': 2,
 'c': 3,
 'p': 4,
 'w': 5,
 'k': 6,
 'z': 7,
 ' ': 8,
 '2': 9,
 ';': 10,
 'j': 11,
 '!': 12,
 "'": 13,
 ':': 14,
 '"': 15,
 'v': 16,
 'n': 17,
 'l': 18,
 'e': 19,
 '.': 20,
 'x': 21,
 'q': 22,
 'o': 23,
 'f': 24,
 '%': 25,
 '/': 26,
 '(': 27,
 '-': 28,
 'm': 29,
 '$': 30,
 'h': 31,
 ',': 32,
 't': 33,
 '=': 34,
 'u': 35,
 'i': 36,
 'g': 37,
 'b': 38,
 '&': 39,
 '?': 40,
 'd': 41,
 'y': 42,
 's': 43,
 ')': 44,
 'a': 45}

In [37]:
# Max len
max_plot_len = 10000
X = [x[:max_plot_len] for x in X]
# Convert characters to int and pad
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in X]

In [38]:
X[5]

[33,
 5,
 23,
 8,
 41,
 1,
 36,
 24,
 33,
 19,
 1,
 43,
 8,
 45,
 1,
 19,
 8,
 4,
 45,
 43,
 43,
 36,
 17,
 37,
 8,
 33,
 31,
 1,
 23,
 35,
 37,
 31,
 8,
 45,
 8,
 5,
 19,
 43,
 33,
 19,
 1,
 17,
 8,
 33,
 23,
 5,
 17,
 8,
 32,
 8,
 8,
 5,
 31,
 19,
 17,
 8,
 17,
 19,
 5,
 43,
 8,
 3,
 23,
 29,
 19,
 43,
 8,
 36,
 17,
 8,
 33,
 31,
 45,
 33,
 8,
 45,
 8,
 18,
 23,
 3,
 45,
 18,
 8,
 24,
 45,
 1,
 29,
 19,
 1,
 8,
 31,
 45,
 43,
 8,
 38,
 19,
 19,
 17,
 8,
 29,
 35,
 1,
 41,
 19,
 1,
 19,
 41,
 8,
 45,
 17,
 41,
 8,
 31,
 36,
 43,
 8,
 3,
 45,
 33,
 33,
 18,
 19,
 8,
 43,
 33,
 23,
 18,
 19,
 17,
 8,
 20,
 8,
 8,
 33,
 31,
 19,
 8,
 33,
 23,
 5,
 17,
 43,
 4,
 19,
 23,
 4,
 18,
 19,
 8,
 32,
 8,
 8,
 11,
 23,
 36,
 17,
 19,
 41,
 8,
 38,
 42,
 8,
 33,
 31,
 19,
 8,
 41,
 1,
 36,
 24,
 33,
 19,
 1,
 43,
 8,
 32,
 8,
 8,
 24,
 23,
 1,
 29,
 8,
 45,
 8,
 4,
 23,
 43,
 43,
 19,
 8,
 33,
 23,
 8,
 3,
 45,
 33,
 3,
 31,
 8,
 33,
 31,
 19,
 8,
 4,
 19,
 1,
 4,
 19,
 33,
 1,
 45,
 33,
 23,
 1,


In [39]:
X_pad = sequence.pad_sequences(X, maxlen=max_plot_len)
X_pad

array([[ 0,  0,  0, ..., 29,  8, 20],
       [ 0,  0,  0, ...,  6,  8, 20],
       [ 0,  0,  0, ...,  5,  8, 20],
       ...,
       [ 0,  0,  0, ..., 41,  8, 20],
       [ 0,  0,  0, ..., 19,  8, 20],
       [ 0,  0,  0, ..., 19,  8, 20]])

In [40]:
#Crear la red
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_pad, y_genres, test_size=0.33, random_state=42)

In [41]:
from keras.layers import Activation, Dense, Flatten 

In [42]:
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_plot_len))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, input_dim=24, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10000, 128)        5888      
_________________________________________________________________
lstm (LSTM)                  (None, 32)                20608     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 26,529
Trainable params: 26,529
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.fit(X_train, y_train_genres, validation_data=[X_test, y_test_genres], 
          batch_size=128, epochs=10, verbose=1,
          callbacks=[PlotLossesKeras()])

Epoch 1/10


ValueError: in user code:

    C:\Users\57310\Anaconda3\lib\site-packages\keras\engine\training.py:830 train_function  *
        return step_function(self, iterator)
    C:\Users\57310\Anaconda3\lib\site-packages\keras\engine\training.py:813 run_step  *
        outputs = model.train_step(data)
    C:\Users\57310\Anaconda3\lib\site-packages\keras\engine\training.py:771 train_step  *
        loss = self.compiled_loss(
    C:\Users\57310\Anaconda3\lib\site-packages\keras\engine\compile_utils.py:201 __call__  *
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    C:\Users\57310\Anaconda3\lib\site-packages\keras\losses.py:142 __call__  *
        losses = call_fn(y_true, y_pred)
    C:\Users\57310\Anaconda3\lib\site-packages\keras\losses.py:246 call  *
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    C:\Users\57310\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    C:\Users\57310\Anaconda3\lib\site-pac